# Setup 

In [2]:
import os
import json
import numpy as np
from tqdm import tqdm
from PIL import Image
from numpy import load
from google.cloud import storage
from google.cloud.storage import transfer_manager

# Extract 

Extract from Kuzushiji dataset

In [11]:
# !unzip ../datasets/kuzushiji.zip -d ../datasets/kuzushiji

In [4]:
labels_train = load("../datasets/kuzushiji/kmnist-train-labels.npz")['arr_0']
img_train = load("../datasets/kuzushiji/kmnist-train-imgs.npz")['arr_0']
labels_test = load("../datasets/kuzushiji/kmnist-test-labels.npz")['arr_0']
img_test = load("../datasets/kuzushiji/kmnist-test-imgs.npz")['arr_0']

# Transform

Transform numpy arrays into PNG images, labels into JSONL

In [13]:
# !mkdir ../datasets/kuzushiji_images
# !mkdir ../datasets/kuzushiji_images/train
# !mkdir ../datasets/kuzushiji_images/test

In [3]:
def export_to_png(npzfile, folder):   
    for i in tqdm(range(npzfile.shape[0])):
        img = Image.fromarray(npzfile[i])
        img.save(os.path.join(folder,"%05d.png" % i))

#export_to_png(img_train, "../datasets/kuzushiji_images/train")
export_to_png(img_test, "../datasets/kuzushiji_images/test")

NameError: name 'img_test' is not defined

## Generate JSONL

In [5]:
def make_rec(bucket, filename, label, phase):
    # https://cloud.google.com/vertex-ai/docs/image-data/classification/prepare-data
    rec = {
        "imageGcsUri": f"gs://{bucket}/{phase}/{filename}",
        "classificationAnnotation": {
            "displayName": label,
            "annotationResourceLabels": {
                "aiplatform.googleapis.com/annotation_set_name": "hiragana",
                "env": "dev"
            }
        },
        "dataItemResourceLabels": {
        "aiplatform.googleapis.com/ml_use": phase
        }
    }
    return rec

train_json = [make_rec("kuzushiji-mnist", "%05d.png" % i, str(int(label)), "train") for i,label in enumerate(list(labels_train)) ]
test_json = [make_rec("kuzushiji-mnist", "%05d.png" % i, str(int(label)), "test") for i,label in enumerate(list(labels_test)) ]

In [8]:
with open("../datasets/kuzushiji/dataset.jsonl", "w") as f:
    for ln in train_json:
        f.write(json.dumps(ln) + "\n")
    for ln in test_json:        
        f.write(json.dumps(ln) + "\n")

# Load

Load into GCP Bucket

In [73]:
if "GOOGLE_APPLICATION_CREDENTIALS" not in os.environ:
    print("GCP credentials are needed for storage bucket")

In [37]:
client = storage.Client()
bucket = client.get_bucket('kuzushiji-mnist')

In [58]:
setnames = ["train","test"]

for setname in setnames:
    root_path = "../datasets/kuzushiji_images/"
    filenames = [fn for fn in os.listdir(root_path + setname) ]
    results = transfer_manager.upload_many_from_filenames(
       bucket, 
       filenames, 
       source_directory= root_path + setname, 
       blob_name_prefix= setname + "/",
       max_workers=1000
    )
    print("Failed:", len([result for result in results if isinstance(result, Exception)]))



Failed: 0
